# Task 1: Sub-Numpy

## Help Functions 

In [13]:
#Within this cell, all validation/exception handling functions as well as other help-functions for the snp-class are implemented


def check_pos_int(x, case):
    """
    Since several functions in this implementation of the subnumpy library take positive integers as input, it makes sense
    to implement the exception handling once in a function to avoid redundancies in the code and make it more modular.
    This function is used for checking the input numbers within tuples. It distinguished between case 1, 2, and 3 to give a more precise
    and descriptive exception string.
    Input:
    Given input x by the user for a function, where a positive integer is expected.
    Case distinguished between check of simple numeric input (1), first element of tuple (2), and second element of tuple (3).
    """
    m_exception = ""
    if (not isinstance(case, int)) or case > 3 or case < 1:
        raise Exception("No such case " + f"{case} " + "second argument must be integer between 1 and 3 inclusively.")
    elif case == 1:
        m_exception = "The argument given ( "
    elif case == 2:
        m_exception = "The first element of the tuple ( "
    elif case == 3:
        m_exception = "The second element of the tuple ( "

    if not isinstance(x, int):
        raise Exception(m_exception + f"{x}" + " ) is no integer.")
    elif isinstance(x, int) and x < 1:
        raise Exception(m_exception + f"{x}" + " ) is an integer smaller than 1.")

def check_tuple(t):
    """
    This function checks the validity of numeric tuples. The input shall be a tuple with two elements and consist of positiv integers.
    For the letter, the fuction check_pos_int is used on both elements of the argument - in case the former is fulfilled.
    """
    if not isinstance(t, tuple):
        raise Exception("The second argument( " + f"{t}" + " )is not a tuple.")
    elif len(t) != 2:
        raise Exception("The second argument( " + f"{t}" + " )is not a tuple of length 2.")
    t0 = t[0]
    t1 = t[1]
    check_pos_int(t0, 2)
    check_pos_int(t1, 3)

def check_arr(v):
    """
    This vector iterates through the vector and checks, if it has only numeric values. Raises exception if requirement not fulfilled.
    """
    dim = len(v)
    for i in range(dim):
        if not(isinstance(v[i], int) or isinstance(v[i], float)):
            raise Exception("The vector ("+ f"{v}" +") does not consist of numeric values only.")

def check_matrix_vector(v_m):
    """
    A matrix/vector is implemented as a list of lists, in which each list represents one row of the matrix/vector.
    To be a valid matrix/vector, v_m must be an list consisting of only numeric lists of which all have the same length.
    """
    if not isinstance(v_m, list):
        raise Exception("The input given is not a matrix - it has the wrong data type.")
    m = len(v_m)
    if not isinstance(v_m, list):
        raise Exception("Wrong input type - shall be list")
    if m < 1:
        raise Exception("Matrix has no values.")
    try:
        n_save = len(v_m[0])
    except:
        raise Exception("Matrix/vector is not a list of lists - wrongly defined.")
    #check that matrix is well defined: each row has same amount of values
    for i in range(m):
        n = len(v_m[i])
        if n != n_save:
            raise Exception("Matrix/Vector is not well defined - shall have same number of row entries for every row.")
    #check that all values in the matrix/vector are numeric ones
    for row in range(m):
        for col in range(n_save):
            if not(isinstance(v_m[row][col], int) or isinstance(v_m[row][col], float)):
                raise Exception("The matrix/vector ("+ f"{v_m}" +") does not consist of numeric values only.")

def swap(matrix, index):
    """
    This function helps to swap rows in respect to the pivot values so that zeros are swapped to the bottom rows.
    It is called for bring rows in the right order regarding the values in the column given as the input argument <<index>>.

    """
    m, n = snp.shape(matrix)
    #the check for leading zeros in a certain column must only be done for the rows down from the index_th row because the row above can be assumed to be in the echelon form already
    for row_idx in range(index, m):
        #the swap must only be carried out if the respective value is equal to zero
        if matrix[row_idx][index] != 0:
            continue
        else:
            #for the swap, a row with a value in the index_th position not equal to zero is searched
            for row_comp in range(row_idx+1, m):
                if matrix[row_comp][index] != 0:
                    #for a potential swap, one row must be saved in another variable
                    row_store = matrix[row_idx]
                    #swap is carried out by exchanging the position of the rows
                    matrix[row_idx] = matrix[row_comp]
                    matrix[row_comp] = row_store
                    break
    return matrix

def reduced_echelon(matrix):
    """
    After the echelon form of the matrix is calculted, this function calculates the reduced echelon form.
    """
    #first, matrix is transformed into the echelon form, which is the first step for the transformation into rref
    m, n = snp.shape(matrix)
    #there is no proper echelon form form if the matrix given as an input is either a row or a column vector
    if m<2 or n<2:
        raise Exception("Matrix that was given as input is too small - both dimesions must be greater than 1.")
    i = 0
    while i < m-1 and i < n:
        matrix = swap(matrix, i)
        #because swap was called before: if ???"anchor"/pivot??? value is zero, echelon is done for that column
        if matrix[i][i] != 0:
            #for "anchor" value, all values below must be set to zero to create the echelon form
            for iter_n in range(i, m-1):
                #if any value under the anchor value is zero, the ones below are as well because swap was called before - next anchor/row can be tackled
                if matrix[iter_n+1][i] == 0:
                    break
                #row operation: row_x = row_x - ( factor * row_(x-1) )
                factor = matrix[iter_n+1][i] / matrix[i][i]
                matrix[iter_n+1] = [a - (factor * b) for a,b in zip(matrix[iter_n+1], matrix[i])]
        i = i + 1
    #to cope with rounding errors that result from float operations, values are rounded to 10 decimal places
    for row in range(m):
        new_row_pre = matrix[row]
        matrix[row] = [round(v, 10) for v in new_row_pre]

    #from standard echelon form, now turn values above diagonal entries to zero
    m, n = snp.shape(matrix)
    #iterating from the lower right to the upper left corner
    i = m - 1
    while i >= 0:
        #if that value is 0, there cannot be a reduction in that column for the rows above based on it.
        if matrix[i][i] != 0:
            for iter_n in range(i, 0, -1):
                if matrix[i][i] == 0:
                    break
                factor = matrix[iter_n-1][i] / matrix[i][i]
                matrix[iter_n - 1] = [a - (factor * b) for a, b in zip(matrix[iter_n - 1], matrix[i])]
        i = i - 1

    #finalizing to rref by deviding row values so that coefficients matrix is diagonal with ones
    for n in range(m):
        if matrix[n][n] != 0:
            value = 1 / ( matrix[n][n] )
            matrix[n] = [value * a for a in matrix[n]]

    #Rounding to cope with floating-point precision errors
    for row in range(m):
        new_row_pre = matrix[row]
        matrix[row] = [round(v, 5) for v in new_row_pre]
    return matrix

def augmented(matrix, vector):
    #validity checks already done in snp.linalg_solver()    
    m, n = snp.shape(matrix)
    n2 = len(vector)
    #there must be one value in the vector for each row of the matrix
    if m != n2:
        raise Exception("Dimension of matrix and vector do not match.")
    for i in range(m):
        matrix[i].append(vector[i])
    return matrix

def solution_finder(matrix):
    """
    This function gets a matrix in rref, for which there is only one solution, as input.
    It gives back the corresponding solution.
    """
    m, n = snp.shape(matrix)
    #case no solution: zero coefficient row and non-zero value in the augmented part
    for row in matrix:
        for i in range(n-1):
            if (all(value == 0 for value in row[:(n-1)]) and (row[n-1] != 0)):
                raise Exception("The system of linear equations does not have a solution.")
            
    #case one solution: 
    for r in range(m):
        for i in range(m):
            #singular matrix exception, if rref (diagonal values non-zero and other values of coefficient matrix zero) was not created
            if ((r == i and matrix[r][i] == 0) or (r != i and matrix[r][i] != 0)):
                raise Exception("Matrix is singular.")
    solution = []
    for i in range(m):
        solution.append(matrix[i][n-1])
    return solution


## snp class - Implementation of Sub-Numpy

In [39]:
class Subnumpy(object):

    #The follwing function is
    def ones(self, x):
        """
        This function creates an list of length x containing only ones.
        Input: x, a positive integer
        Output: list of length x containing only ones
        """
        check_pos_int(x, 1)
        out = []
        out = [1 for i in range(x)]
        return out

    def zeros(self, x):
        """
        This function creates an list of length x containing only zeros.
        Input: x, a positive integer
        Output: list of length x containing only zeros
        """
        check_pos_int(x, 1)
        out = []
        out = [0 for i in range(x)]
        return out

    def reshape(self, arr, t):
        """
        This function creates a matrix/vector of the dimension inputted with the tuple out of the values in the list.
        Important is to check for coherence of the list and the number of its entries with the dimension given.
        """
        check_arr(arr)
        check_tuple(t)
        m = t[0]
        n = t[1]
        matrix = []
        if m*n != len(arr):
            raise Exception("Length of vector and matrix dimension do not match.")
        else:
            for m_rows in range(m):
                x1 = m_rows * n
                x2 = (m_rows + 1) * n
                matrix.append(arr[x1:x2])
            return matrix

    def shape(self, matrix):
        """
        This function returns the dimensions m and n of a mxn matrix.
        Before calculating, it has to be checked if the matrix-requirements for the input sre fulfilled.
        Input: Valid matrix.
        Output: Dimensions m and n of mxn matrix
        """
        check_matrix_vector(matrix)
        m = len(matrix)
        n = len(matrix[0])
        return (m, n)

    def append(self, matrix1, matrix2, axis=None):
        """
        There are three cases for this function.
        axis=None: all values given as input are concenated into one list
        axis=0: the second matrix is concatenated below the first one. Number of columns have to match for both matrices.
        axis=1: the second matrix is concatenated right of the first one. Number of rows have to match for both matrices.
        """
        m1, n1 = snp.shape(matrix1)
        m2, n2 = snp.shape(matrix2)
        res = []
        #axis=None: all numeric values are appended into one list from first to second matrix and from the first value of the first from to the last value of the last row.
        if axis == None:
            for row1 in matrix1:
                for elem1 in row1:
                    res.append(elem1)
            for row2 in matrix2:
                for elem2 in row2:
                    res.append(elem2)
        #axis=0: matrix2 is appended below matrix1
        elif axis == 0:
            if n1 != n2:
                raise Exception("All the input list dimensions for the concatenation axis must match exactly, but along dimension 0.")
            res = matrix1
            for row in matrix2:
                res.append(row)
        #axis=1: matrix2 is appended on the right of matrix1
        elif axis == 1:
            if m1 != m2:
                raise Exception("All the list list dimensions for the concatenation axis must match exactly, but along dimension 1.")
            res = matrix1
            for i in range(m1):
                for n in range(n2):
                    matrix1[i].append(matrix2[i][n])
        else:
            raise Exception("Axis, the third parameter is out of bounds - must be None, 1, or 2.")
        return res
            

    def get(self, v_m, dim):
        """
        This function returns the values pecified by the coordinate point (row,column) of the list provided
        (can be vector or matrix).
        Input: Valid Vector or Matrix and a valid tuple specifiying the coordinate of the value of interest.
        Note that the input must be given in natural numbers starting with (1, 1) for the top left entry.
        Output: Numeric value
        """
        check_matrix_vector(v_m)
        m, n = snp.shape(v_m)
        check_tuple(dim)
        if (dim[0]) > m or (dim[1]) > n:
            raise Exception("Dimension input does not exists for the given matrix/vector.")
        else:
            dim1_pre, dim2_pre = dim
            dim1 = dim1_pre - 1
            dim2 = dim2_pre - 1
            value = v_m[dim1][dim2]
        return value

    def add(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if m1 != m2 or n1 != n2:
            raise Exception("Operation not possible - both matrices/vectors given do not have the same dimesnsions.")
        result = []
        for i in range(0,m1):
            result.append([a + b for a, b in zip(v_m1[i], v_m2[i])])
        return result

    def subtract(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if m1 != m2 or n1 != n2:
            raise Exception("Operation not possible - both matrices/vectors given do not have the same dimesnsions.")
        result = []
        for i in range(0,m1):
            result.append([a - b for a, b in zip(v_m1[i], v_m2[i])])
        return result

    def dotproduct(self, v_m1, v_m2):
        check_matrix_vector(v_m1)
        check_matrix_vector(v_m2)
        m1, n1 = snp.shape(v_m1)
        m2, n2 = snp.shape(v_m2)
        if n1 != m2:
            raise Exception("The combination of the input vectors/matrices do not fulfill the requirements for the dotprouct.\n     The number of rows in the first input must be equal to the number of columns of the second input.")
        #initialize resulting matrix/vector
        res_dim = m1, n2
        res = []
        zero_row = snp.zeros(n2)
        for i in range(m1):
            res.append(zero_row)
        for m in range(m1):
            for n in range(n2):
                sum = 0
                for iterate in range(n1):
                    sum = sum + ( v_m1[m][iterate] * v_m2[iterate][n] )
                res[m][n] = sum
        return res

    def linalg_solver(self, matrix, vector):
        check_matrix_vector(matrix)
        #since the input of the vector in numpy is implemented unintuitively as normal array - and we want to do it analogous - an adjusted error message is helpful if the input for specifying the issue for the user.
        try:
            check_arr(vector)
        except:
            raise Exception("The vector has to be given as a normal list - not a vector.")
        m1, n1 = snp.shape(matrix)
        n2 = len(vector)
        #check for coherence of matrix and vector - must be same number of rows and vector must only have one column
        if (m1 != n2):
            raise Exception("The dimension of the input - matrix & vector - do not fit.")
        if (m1 != n1):
            raise Exception("Matrix must be square.") #np.linalg.solver has same requirement
        #create the augmented matrix - as implemented in np.linalg.solve - the vector must be given in the format [x, y, ...]
        aug_matrix = augmented(matrix, vector)
        rref = reduced_echelon(aug_matrix)
        print(rref)
        return solution_finder(rref)


In [40]:
snp = Subnumpy()
x = snp.zeros(10)

In [41]:
matrix1 = [[1, 2], [3, 4]]
vector1 = [5, 6]

matrix2 = [[2, 4], [1, 3]]
vector2 = [7, 8]

matrix3 = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
vector3 = [2, 3, 4]

matrix4 = [[5, 7, 2], [1, 8, 3], [4, 6, 9]]
vector4 = [1, 0, 2]

matrix5 = [[2, 5, 1, 4], [3, 8, 6, 7], [9, 4, 2, 1], [6, 7, 3, 8]]
vector5 = [3, 2, 1, 4]

matrix6 = [[4, 7, 9], [1, 5, 3], [8, 2, 6]]
vector6 = [5, 6, 7]

matrix7 = [[2, 1], [1, 2]]
vector7 = [8, 9]

matrix8 = [[3, 6, 9], [12, 15, 18], [21, 24, 27]]
vector8 = [1, 0, 9]

matrix9 = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
vector9 = [4, 3, 2, 1]

matrix10 = [[3, 2, 1], [6, 5, 4], [9, 8, 7]]
vector10 = [5, 2, 8]

matrix11 = [[1, 2], [3, 4]]
vector11 = [-1, 2]

matrix12 = [[5, 2], [8, 3]]
vector12 = [7, 1]

matrix13 = [[1, 0], [0, 1]]
vector13 = [6, 5]

matrix14 = [[4, 7, 1], [2, 9, 8], [6, 3, 5]]
vector14 = [3, 2, 1]

matrix15 = [[2, 3, 5, 7], [11, 13, 17, 19], [23, 29, 31, 37], [41, 43, 47, 53]]
vector15 = [8, 6, 4, 2]

In [42]:
import numpy as np
x = np.linalg.solve(np.array(matrix15), np.array(vector15))
x

array([ 0.43636364, -2.92727273, -1.14545455,  3.09090909])

In [43]:
x = snp.linalg_solver(matrix15, vector15)
x

[[1.0, 0.0, 0.0, 0.0, 0.43636], [-0.0, 1.0, -0.0, -0.0, -2.92727], [-0.0, -0.0, 1.0, -0.0, -1.14545], [0.0, 0.0, 0.0, 1.0, 3.09091]]


[0.43636, -2.92727, -1.14545, 3.09091]

In [50]:
x = [1,2,3,4,5,6,7,8,9]
v = snp.reshape(x, (3,3))


y = [1,0,1]
#uzz = snp.dotproduct(, v)
#uzz

In [61]:
x = [1,2,3,4,5,6,7,8,9]
v = snp.reshape(x, (3,3))


y = [1,0,1]
snp.linalg_solver(v, y)

[[1.0, 0.0, -1.0, -1.66667], [-0.0, 1.0, 2.0, 1.33333], [0.0, 0.0, 0.0, 2.0]]


Exception: The system of linear equations does not have a solution.

In [60]:
x = [1,2,3,4,5,6,7,8,9]
v = snp.reshape(x, (3,3))

y = [1,0,1]
np.linalg.solve(np.array(v), np.array(y))

array([-9.00719925e+15,  1.80143985e+16, -9.00719925e+15])